In [150]:
import csv
import os

import pandas as pd

class ExtractData:
    def __init__(self, file_in):
        self.file_in = file_in
        self.file_out = None
    
    @staticmethod
    def _get_company_name(file_in):
        reader = pd.read_excel(file_in, sheet_name='1000000')
        dict_temp = {}

        for i in range(len(reader)):
            key = reader['Unnamed: 2'][i]
            val = reader['Unnamed: 1'][i]
            dict_temp[key] = [val]
        
        df = pd.DataFrame(dict_temp)
        df = df.rename(columns={'Entity code': 'entity_code'})

        # Use __getitem__ method, aka. [[...]] to return type DataFrame, instead of series
        return df[['entity_code']]
        
    def general_inf(self):
        file_in = self.file_in
        self.file_out = '/content/drive/MyDrive/Program/Python/Data ML DL/Data Cleaning and Analyzing/IDX - Financial Statements Cleanup/company_general_information.csv'
        
        # index_col=0 is to remove auto-generated index column
        gen_inf = pd.read_excel(file_in, sheet_name='1000000', index_col=0)
        gen_inf_dict = {}
        
        for i in range(len(gen_inf)):
            key = gen_inf['Unnamed: 2'][i]
            val = gen_inf['Unnamed: 1'][i]
            gen_inf_dict[key] = [val]
        
        df = pd.DataFrame(gen_inf_dict)

        # Rename the first dataframe header
        df.columns.values[0] = 'date_of_report'
        df = df.rename(columns={
            'Entity name': 'entity_name',
            'Entity code': 'entity_code',
            'Explanation of change in name from the end of the preceding reporting period': 'name_change_explanation',
            'Entity identification number': 'identification_number',
            'Entity main industry': 'main_industry',
            'Controlling shareholder information': 'information_control',
            'Type of entity': 'entity_type',
            'Type of listed securities': 'securities_type',
            'Type of board on which the entity is listed': 'type_of_board',
            'Whether the financial statements are of an individual entity or a group of entities': 'statements_from',
            'Period of financial statements submissions': 'period',
            'Current period start date': 'start_date',
            'Current period end date': 'end_date',
            'Prior period start date': 'prior_start_date',
            'Prior period end date': 'prior_end_date',
            'Description of presentation currency': 'currency',
            'Conversion rate at reporting date if presentation currency is other than rupiah': 'alternate_currency',
            'Level of rounding used in financial statements': 'rounding_level',
            'Type of report on financial statements': 'report_type',
            'Type of auditor\'s opinion': 'auditor_opinion_type',
            'Matters disclosed in emphasis-of-matter or other-matter paragraph, if any': 'emphasis_of_matter',
            'Result of review engagement': 'review_result',
            'Date of auditor\'s opinion or result of review report': 'date_of_review',
            'Name of current year audit signing partner': 'signing_partner_name',
            'Number of years served as audit signing partner': 'signing_partner_experience',
            'Name of prior year audit signing partner': 'prior_year_signing_partner',
            'Whether in compliance with BAPEPAM LK VIII G 11 rules concerning responsibilities of board of directors on financial statements': 'BAPEPAM_LK_VIII_G11',
            'Whether in compliance with BAPEPAM LK VIII A two rules concerning independence of accountant providing audit services in capital market': 'BAPEPAM_LK_VIII_A2',

            'Sector': 'sector',
            'Subsector': 'subsector',
            'Prior year end date': 'prior_year_end',
            'Current year auditor': 'current_year_auditor',
            'Prior year auditor': 'prior_year_auditor',
        })
        df = df.drop(columns=['General information'])

        if os.path.exists(self.file_out):
            read_file_out = pd.read_csv(self.file_out)
            if df['entity_code'][0] not in list(read_file_out['entity_code']):
                # mode = 'a' is to append, default is 'w' to write
                df.to_csv('./company_general_information.csv', mode='a', header=False, index=False)

            for j in range(len(read_file_out)):
                # Use .iloc[] so the DataFrame can accept negative index
                if (
                    df['entity_code'][0] == read_file_out['entity_code'].iloc[j] and
                    df['end_date'][0] != read_file_out['end_date'].iloc[j]
                ):
                    df.to_csv('./company_general_information.csv', mode='a', header=False, index=False)
        else:
            df.to_csv('./company_general_information.csv', index=False)
    
    def financial_pos(self):
        fin_pos = pd.read_excel(self.file_in, sheet_name='1210000', index_col=0)
        self.file_out = '/content/drive/MyDrive/Program/Python/Data ML DL/Data Cleaning and Analyzing/IDX - Financial Statements Cleanup/company_financial_position.csv'

        header = []
        content = []
        fin_dict = {}

        df_get_name = self._get_company_name(self.file_in)

        for i in range(len(fin_pos)):
            key = fin_pos['Unnamed: 3'][i]
            val = fin_pos['Unnamed: 1'][i]
            # The val need to be encapsulated inside a list
            fin_dict[key] = [val]

        df_temp = pd.DataFrame(fin_dict)
        df_temp.columns.values[0] = 'date_of_report'

        # Slice the dataframe with __getitem__ method aka. [[...]]
        df_trim = df_temp[[
                'date_of_report', 'Total current assets', 'Total non-current assets', 'Total assets',
                'Total current liabilities', 'Total non-current liabilities', 'Total liabilities',
                'Total equity', 'Total liabilities and equity',
        ]]
        df_trim = df_trim.rename(
            columns = {
                'Total current assets': 'total_current_assets',
                'Total non-current assets': 'total_noncurrent_assets',
                'Total assets': 'total_assets',
                'Total current liabilities': 'total_current_liabilities',
                'Total non-current liabilities': 'total_noncurrent_liabilities',
                'Total liabilities': 'total_liabilities',
                'Total equity': 'total_equity',
                'Total liabilities and equity': 'total_liabilities_and_equity',
            }
        )

        # axis=1 was to combine the DataFrames according to its index,
        # instead of making new row for each data inside the DataFrames like the default behaviour
        df = pd.concat([df_get_name, df_trim], axis=1)

        if os.path.exists(self.file_out):
            read_file_out = pd.read_csv(self.file_out)

            if df['entity_code'][0] not in list(read_file_out['entity_code']):
                # mode = 'a' is to append, default is 'w' to write
                df.to_csv('./company_financial_position.csv', mode='a', header=False, index=False)

            for j in range(len(read_file_out)):
                # Use .iloc[] so the DataFrame can accept negative index
                if (
                    df['entity_code'][0] == read_file_out['entity_code'].iloc[j] and
                    df['end_date'][0] != read_file_out['end_date'].iloc[j]
                ):
                    df.to_csv('./company_financial_position.csv', mode='a', header=False, index=False)
        else:
            df.to_csv('./company_financial_position.csv', index=False)

file_in_1 = '/content/drive/MyDrive/Program/Python/Data ML DL/Datasets/IDX - Financial Statements/ABBA/FinancialStatement-2021-I-ABBA.xlsx'
file_in_2 = '/content/drive/MyDrive/Program/Python/Data ML DL/Datasets/IDX - Financial Statements/AALI/FinancialStatement-2021-II-AALI.xlsx'
init = ExtractData(file_in_2)
init.financial_pos()

In [151]:
!cp company_financial_position.csv "/content/drive/MyDrive/Program/Python/Data ML DL/Data Cleaning and Analyzing/IDX - Financial Statements Cleanup"

In [134]:
import pandas as pd
import csv
import os

FILENAME = "/content/drive/MyDrive/Program/Python/Data ML DL/Datasets/IDX - Financial Statements/AALI/FinancialStatement-2021-II-AALI.xlsx"
fin_pos = pd.read_excel(FILENAME, sheet_name='1210000', index_col=0)
header = []
content = []
fin_dict = {}

for i in range(len(fin_pos)):
    key = fin_pos['Unnamed: 3'][i]
    val = fin_pos['Unnamed: 1'][i]
    # The val need to be encapsulated inside a list
    fin_dict[key] = [val]

df = pd.DataFrame(fin_dict)
df.columns.values[0] = 'date_of_report'

df_trim = df[[
        'date_of_report', 'Total current assets', 'Total non-current assets', 'Total assets',
        'Total current liabilities', 'Total non-current liabilities', 'Total liabilities',
        'Total equity', 'Total liabilities and equity',
]]

CSV_FILE = '/content/drive/MyDrive/Program/Python/Data ML DL/Data Cleaning and Analyzing/IDX - Financial Statements Cleanup/company_general_information.csv'

read_csv = pd.read_csv(CSV_FILE)

,date_of_report,Total current assets,Total non-current assets,Total assets,Total current liabilities,Total non-current liabilities,Total liabilities,Total equity,Total liabilities and equity
0,30 June 2021,7629855,21059167,28689022,2009568,6732049,8741617,19947405,28689022


# Using `pandas` Library
Before export it to `.csv`, we need to convert the data into `pandas` `DataFrame` format.

In [ ]:
import pandas as pd
import csv
import os

FILENAME = "/content/drive/MyDrive/Program/Python/Data ML DL/Datasets/IDX - Financial Statements/AALI/FinancialStatement-2021-I-AALI.xlsx"
general_inf = pd.read_excel(FILENAME, sheet_name='1000000', index_col=0)
general_inf_header = []
general_inf_content = []
general_inf_dict = {}

for i in range(len(general_inf)):
    key = general_inf['Unnamed: 2'][i]
    val = general_inf['Unnamed: 1'][i]
    # The val need to be encapsulated inside a list
    general_inf_dict[key] = [val]

df = pd.DataFrame(general_inf_dict)
CSV_FILE = '/content/drive/MyDrive/Program/Python/Data ML DL/Data Cleaning and Analyzing/IDX - Financial Statements Cleanup/company_general_information.csv'

if os.path.exists(CSV_FILE):
    res_csv = pd.read_csv(CSV_FILE)

    for j in range(len(res_csv)):
        # Use .iloc[] so the DataFrame can accept negative index
        if df['Entity code'][0] != res_csv['Entity code'].iloc[-1]:
            # mode='a' means to 'append', default was 'w' as in 'write'
            df.to_csv('company_general_information.csv', mode='a', header=False)
        elif df['Entity code'][0] != res_csv['Entity code'].iloc[j]:
            df.to_csv('company_general_information.csv', mode='a', header=False)
            break
        elif (
            df['Entity code'][0] == res_csv['Entity code'].iloc[j] and
            df['Current period start date'][0] != res_csv['Current period start date'].iloc[j]
        ):
            df.to_csv('company_general_information.csv', mode='a', header=False)
            break
else:
    df.to_csv('company_general_information.csv')

In [ ]:
!cp company_general_information.csv "/content/drive/MyDrive/Program/Python/Data ML DL/Data Cleaning and Analyzing/IDX - Financial Statements Cleanup"

# Alternative Way To Do It
We're utilizing Python's standard libary `csv` here instead of `pandas`.

In [ ]:
FILENAME = "/content/drive/MyDrive/Program/Python/Data ML DL/Datasets/IDX - Financial Statements/AALI/FinancialStatement-2021-I-AALI.xlsx"
general_inf = pd.read_excel(FILENAME, sheet_name='1000000')
general_inf_header = []
general_inf_content = []
general_inf_dict = {}

for i in range(len(general_inf)):
    general_inf_header.append(general_inf['Unnamed: 2'][i])
    general_inf_content.append(general_inf['Unnamed: 1'][i])

with open('text.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(general_inf_header)
    writer.writerow(general_inf_content)

In [ ]:
!cp text.csv "/content/drive/MyDrive/Program/Python/Data ML DL/Data Cleaning and Analyzing/IDX - Financial Statements Cleanup"

In [ ]:
general_inf_dict = {}

for i in range(len(general_inf)):
    key = general_inf['Unnamed: 2'][i]
    val = general_inf['Unnamed: 1'][i]
    # The val need to be encapsulated inside a list
    general_inf_dict[key] = [val]

df = pd.DataFrame(general_inf_dict)
df['Entity code'].iloc[-1]

'AALI'